In [ ]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Prose/Mathieu/Benter-Project

In [2]:
%env ENVIRONMENT=development

In [4]:
import collections

from models.horse import Horse
from models.runner import Runner
from models.race import Race
from database.setup import create_sqlalchemy_session
import tqdm
import random
import re
from tabulate import tabulate
import os
import json
import datetime as dt
from typing import List
import sqlalchemy as sa
from collections import Counter
from  sqlalchemy.sql.expression import func
import pandas as pd
from Levenshtein import distance as lev_distance
from scipy.spatial.distance import pdist
import numpy as np
from pprint import pprint

from utils.music import parse_music, parse_unibet_music
from utils.logger import setup_logger
from constants import UnibetHorseSex

BATCH_SIZE = 1e5
UNIBET_DATA_PATH = "./data/Unibet"
#with create_sqlalchemy_session() as db_session:
#    horse=db_session.query(Horse).first()
    
logger = setup_logger(__name__)
    

In [5]:
names=pd.read_csv("./data/persons_202101180829.csv")

In [6]:
names.head()

,id,name
0,263819,D'ersu J.
1,263820,Bertran De Balanda J.
2,263821,Chameraud B.
3,263822,Mlle Isabelle Gallorini
4,263823,Gallorini Mlle I.


In [7]:
names=set(names['name'].to_list())

In [8]:
len(names)

154206

In [9]:
names=sorted(list(names))

In [10]:
for name in names:
    if not re.search(r"\w", name):
        print(name)

********************
-
.
...............


In [11]:
names=[name for name in names if re.search(f'\w', name)]

In [12]:
names_=[]
for name in names:
    
    if re.match(r"^\W.*", name):
        name = re.sub(r'^\.', "", name).strip()
        names_.append(re.sub(r'^\.', "", name).strip())
    else:
        names_.append(name)
        
names=names_
del names_

In [13]:
name=random.choice(names)
name='Morton'
distance_dict = {n: lev_distance(n, name) for n in names if n !=name}

print(name)
print()

pprint(sorted(distance_dict.items(), key=lambda i: i[1])[:40])

Morton

[('D Morton', 2),
 ('Martin', 2),
 ('Mor An', 2),
 ('Morotin', 2),
 ('Morton T', 2),
 ('Boutin', 3),
 ('Coheton', 3),
 ('F Norton', 3),
 ('For An', 3),
 ('Foran', 3),
 ('Fortuna', 3),
 ('Fortune', 3),
 ('Iaron', 3),
 ('M Ito', 3),
 ('Maiten', 3),
 ('Mar An', 3),
 ('Marmaon', 3),
 ('Martens', 3),
 ('Martes', 3),
 ('Martina', 3),
 ('Martino', 3),
 ('Martul', 3),
 ('Mentor', 3),
 ('Mirko', 3),
 ('Mon Kn', 3),
 ('Monin', 3),
 ('Montana', 3),
 ('Monter', 3),
 ('Mor Em', 3),
 ('Mor Ev', 3),
 ('Mor Gu', 3),
 ('Mor Ha', 3),
 ('Mor Ir', 3),
 ('Mor Pi', 3),
 ('Mor Th', 3),
 ('Morat', 3),
 ('Morautos', 3),
 ('Moroco', 3),
 ('Morton D.', 3),
 ('Mott J', 3)]


In [14]:
re.findall(r'[A-Z]\.', name)

[]

In [15]:
re.findall(r'\w{2,}', name)

['Morton']

In [16]:
[ n for n in names if n!= name and set(re.findall(r'\w{2,}', name))==set(re.findall(r'\w{2,}', n))]

['A L R Morton',
 'A.L.R.  Morton',
 'D L Morton',
 'D Morton',
 'Morton D.',
 'Morton T']

In [17]:
[ n for n in names if n!= name and re.search(r'\bMorton\b', n)]

['A L R Morton',
 'A.L.R.  Morton',
 'Brian Morton & Pinnacle Dream Partnership',
 'D L Morton',
 'D Morton',
 'Daniel Morton',
 'Esler, Kristen L., Esler, Matt and Morton, Tobey L.',
 'John Morton',
 'John Morton & Andrew Sharpe',
 'Michael Morton',
 'Morton D.',
 'Morton T',
 'Morton, Tobey, L.',
 'Mr A L R Morton',
 'Mr Glenn Barrie Morton & Mrs Lisa Anne Morton',
 'Mr John Morton',
 'Mrs C L Morton',
 'Mrs C L Morton, F F Fairthorne (',
 'Mrs M N Morton',
 'Rachel Morton',
 'Richard Morton',
 'Tobey L. Morton',
 'Tobey L. Morton & Chuck Hovitz',
 'Tobey Morton']

In [18]:
len([name for name in names if name.count("&")>1])

1261

In [19]:
len([name for name in names if name.count("&")>0])

9889

In [20]:
len(names)

154202

In [21]:
len([name for name in names if name.count(",")>0])

4741

In [22]:
len([name for name in names if name.count("-")>0])

5362

In [253]:
ParsedName=collections.namedtuple("ParsedName", ['person_names', 'corporation_names'])
corporation_marker = {'Co ', 'Inc', 'Ltd', 'Partner', 'partner', 'LLC', 'Ecurie', 'ecurie', 
                      'corp', 'Corp.', 'corp.', 'Llc', 'Gmbh', 'Limited', 'Syndicate', 'Team', 'team', 'Racing', 'Horseracing'
                   'Developments', 'Family', 'family', 'Plc', 'plc', 'ec.'}
regex_separator_marker = r'(?<!\b\w)(?<!\b\w\s)(?<!^)(?<!^\s)(?<!Ch|mr)(?<!(?:Ch|mr)\s)(?:[&\-\/&,]|\sAnd\s|\set\s)(?!\s?\w\b)(?!\s?Inc)(?!\s?Partner)(?!\s?Co)'
forbidden_beginnings = [r'\b\w', '^', 'Ch', 'Mr', 'M']
forbidden_endings = [r'\w\b', 'Inc', 'Partner', 'Co', 'Corp', 'Jr', '$']
separators = "[&-\/&,]|\sAnd\s|\set\s"
regex_separator = "".join(f"(?<!{b})(?<!{b}\s)" for b in forbidden_beginnings)
regex_separator +=f"(?:{separators})"
regex_separator += f"(?!\s?(?:{'|'.join(forbidden_endings)}))"
print(regex_separator)
def parse_name(name:str):
    """Returns 
    - person names
    - corporation names"""
    
    person_names, corporation_names = [], []
    
    entities = re.split(regex_separator, name, flags=re.IGNORECASE)
    
    for entity in entities:
        if entity is None:
            continue
        if entity.strip() == '':
            continue
        if re.search("|".join(corporation_marker), entity, flags=re.IGNORECASE):
            corporation_names.append(entity.strip())
        else:
            person_names.append(entity.strip())
            
    return ParsedName(person_names=person_names, corporation_names=corporation_names)

(?<!\b\w)(?<!\b\w\s)(?<!^)(?<!^\s)(?<!Ch)(?<!Ch\s)(?<!Mr)(?<!Mr\s)(?<!M)(?<!M\s)(?:[&-\/&,]|\sAnd\s|\set\s)(?!\s?(?:\w\b|Inc|Partner|Co|Corp|Jr|$))


In [254]:
def retrieve_person_names(names):
    res=set()
    for name in names:
        parsed_name=parse_name(name)
        res.update(parsed_name.person_names)
        
    return res


def retrieve_company_names(names):
    res=set()
    for name in names:
        parsed_name=parse_name(name)
        res.update(parsed_name.corporation_names)
        
    return res

In [255]:
len(retrieve_person_names(names))

149813

In [256]:
random.sample(retrieve_person_names(names), 20)

['Clements Ms.',
 'Chantal Kusters',
 'Fernando  Martinez Zubizarreta',
 'Lelievre A.',
 'bentall',
 'Nom: Mr W & Mrs N Ackerman)',
 'Ronchalon / G.Augustin',
 'Kje Mi',
 'Korsman M. E.',
 'J.M.R.',
 'Agf',
 'Ferron E.',
 'Gauffenic R.',
 "Thomas O'flynn",
 'Sinnott D.',
 'Hermaize C',
 'Graciela E. Gonzalez',
 'Lu Tseng Chi',
 'M Massimiliano Monte',
 'H. Müller']

In [257]:
len(retrieve_company_names(names))

12151

In [258]:
random.sample(retrieve_company_names(names), 20)

['Mme Decourcelle',
 'Favourites Racing Syndicate',
 'The Cristal Racing Partnership',
 'Ecurie Nunzio Minopoli',
 "Lane's End Racing",
 'Mei Hao Anhui Syndicate',
 'Wood Street Syndicate Ii',
 'My Meadowview Farm LLC',
 'Marco Andres Morales',
 'Ecurie Mnwlkr Trotting Ab',
 'Farmhouse Holdings Pty Ltd',
 'Patrick Burling Developments Ltd',
 'Racing Club Bruno Bard',
 'The Secretly Hopeful Partnership',
 'Fraternity Syndicate',
 'Ecurie Hico B.T',
 'Stud Rincon Del Cerro',
 'Plym Ab Ecurie Fargprodukter',
 'Ecurie Des Omblais',
 'Irish Charm Thoroughbreds LLC']

In [259]:
persons={}
for name in names:
    parsed_name=parse_name(name)
    for p_name in parsed_name.person_names:
        if p_name not in persons:
            persons[p_name]=[name]
        else:
            persons[p_name].append(name)


In [260]:
selected_p_names =random.sample(list(persons), 20)
{k:v for k,v in persons.items() if k in selected_p_names }

{'Abrivard L.C': ['Abrivard L.C'],
 'Bantam G.': ['Bantam G.'],
 'Bovay Marc': ['Bovay Marc-Andre',
  'Bovay Marc-André',
  'Bovay Marc-andre',
  'Bovay Marc-andré'],
 'Brendan Mccarthy': ['Brendan Mccarthy'],
 'Cummings B.': ['Cummings B.'],
 'Derycke P.': ['Derycke P.'],
 'G.Flugge': ['G.Flugge'],
 'Gerard  Hanquiez': ['Gerard  Hanquiez'],
 'Heitmann C.': ['Heitmann C.'],
 'Ivette Perez Chambless': ['Ivette Perez Chambless'],
 'J. L. Torres': ['J. L. Torres'],
 'Klesaris Sb.': ['Klesaris Sb.'],
 'Lemonnier Mlle J.': ['Lemonnier Mlle J.'],
 'Mclean J.': ['Mclean J.'],
 'Mrs Barbara Coakley': ['Mrs Barbara Coakley'],
 'Mulrennan P.': ['Mulrennan P.'],
 'Per Ha': ['Per Ha'],
 'Lacaille Mme M.': ['Scandella-Lacaille Mme M.'],
 'Vebostad': ['Vebostad-trudvang Mme L.', 'Vebostad-trudvang Mme Lou.'],
 'Yeung Shing Fung': ['Yu Kwok Leung & Yeung Shing Fung']}

In [261]:
{p:persons[p] for p in sorted(persons, key=len)[:100]}

{'*': ['*. Vives/m. Delaunay           L',
  'Disp.Art.27§viii**',
  'Disp.Art.27§viii***'],
 '3': ['3'],
 'A': ['A'],
 '.': ['Anastasi/ukegawa/vaughn Rac. .',
  'Anstu Stables Inc. .',
  'Centaur Farms Inc. .',
  'D.S.M. Trotting ..',
  'De Groote Lieven Bvba ..',
  'Del Team E.(.',
  'Euro-trotting ..',
  'Everest Stables Inc. .',
  'Fluxa JME..',
  'Fox Hill Farms Inc. .',
  'Fox Ridge Farm Inc. .',
  'Godolphin Racing Inc. .',
  'Horton Stable Inc. .',
  'Ktwo Partners ..',
  'Lucky Field Inc. .',
  'Matic V.O.F. ..',
  'Melnyk Racing Stables Inc. .',
  'Phipps Stable Corp. .',
  'Razza Dormello Olgiata (citai) .',
  'Roy F Mlle..',
  'Silverleaf Farms Inc. .',
  'Stall Lucky Stables Internat. .',
  'Star Breeding ..',
  'Stronach Stables Inc. .',
  'Tnt Stud/stonewall Farm Stall. .',
  'Union Stable ..',
  'Vervloet Bvba ..',
  'X...'],
 'B': ['B'],
 '&': ['Claiborne Farm (seth Hancock) &',
  'Fern Circle Stables (fireman) &',
  'Michael Tabor, Derrick Smith, &',
  'Teresa Viola R

In [262]:
{p:persons[p] for p in sorted({k:v for k,v in persons.items() if len(k)==5}, key=len)[:10]}

{'Abela': ['Abela (aus)', 'Abela', 'Abela (nz)'],
 'Baree': ['Baree (bel)'],
 'Braun': ['Braun'],
 'Crews': ['Crews (aus)'],
 'Crone': ['Crone'],
 'Grijp': ['Grijp (bel)', 'Grijp'],
 'Sagaj': ['Sagaj (svn)', 'Sagaj'],
 'bel )': ['Van Der Biest (bel )'],
 'Welsh': ['Welsh'],
 'Frere': ['Wertheimer & Frere',
  'Wertheimer & Frere',
  'Wertheimer And Frere',
  'Wertheimer Et Frere (alain & Ger',
  'Wertheimer and Frere']}

In [263]:
pb_persons=set()
for p in persons:
    if len(p)<=4:
        pb_persons.update(persons[p])
len(pb_persons)

6036

In [264]:
pb_persons

{'Sc. Tibi/c. Menini',
 'Ackermann.Bo-ga.Westphal',
 '07/08 Andy T W Leung Trainer Syndicate',
 'Kuku',
 'Mme A.Deregnaucourt/jcn.Bignon',
 'Kantek Petr, Mvdr',
 'Gh.Rogerie / J.Lodico',
 'Messrs M A Currie, C N Francis, B G Gardner, Jarryd Lahoud, Allan F Peters, Larr',
 'Jean-vincent  Toux',
 'Sarl . Ecurie Tagada',
 'Tall Papillon (swi)',
 'Bf. Boutboul/s. Puglia',
 'Middleham Park Racing Lxxii & Pa',
 'Ec. Loic Appere',
 'Sr. Pulido M.',
 'Aa. Neveu',
 'Th.Gries',
 'St. Johns',
 'Cud. Vayla Di Vico ( Ity)',
 'Mr C J H Van Niekerk & Portapa (pty) Ltd T/a Supabets (nom: Mr P Anastassopolous',
 'Ec. Haspel/haras Du Logis',
 'A.Foulon/ec.Vivaldi',
 'Mary-mar (ro)',
 'Res3',
 '08/09 Sean Woods Trainer Syndicate',
 'Ec. GALOETRO S.A.S.',
 'Acl',
 'Jc.Bignon / D.Lassaussaye',
 'Mmw & Premier Racing Stable',
 'Jean-pierre  Philippot',
 'Suc.  De Moratalla',
 'Hs. La Aldea',
 'F. Serre/h.Hayoun/bh.Levesque',
 'Eriksson Alexandra & Leif',
 'Karlsson Ann-sofi',
 'Hs. Las Casuarinas',
 'K. Bec

In [265]:
corporations={}
for name in names:
    parsed_name=parse_name(name)
    for c_name in parsed_name.corporation_names:
        if c_name not in corporations:
            corporations[c_name]=[name]
        else:
            corporations[c_name].append(name)


In [266]:
selected_c_names =random.sample(list(corporations), 20)
{k:v for k,v in corporations.items() if k in selected_c_names }

{'Al Tamira Racing Stable': ['Al Tamira Racing Stable & David A Bernsen'],
 'Anchordoqui Recondo J.': ['Anchordoqui Recondo J.'],
 'Byerley Team': ['Byerley Team'],
 'Clover Racing Stable': ['Clover Racing Stable'],
 'Del Mar Collection': ['Del Mar Collection'],
 'Desfrieches H.': ['Desfrieches H.'],
 'Ecurie Du Graal': ['Ecurie Du Graal'],
 'Ecurie Oberpinzgau': ['Ecurie Oberpinzgau'],
 'Ecurie Vallin .': ['Ecurie Vallin .'],
 'Eden Park Racing Stable': ['Eden Park Racing Stable'],
 'G.G.Syndicate Ltd/o.G.Glenn': ['G.G.Syndicate Ltd/o.G.Glenn'],
 'Forbes And Partner': ['Klein, Forbes And Partner'],
 'Only One Bid Partnership': ['Only One Bid Partnership'],
 'Robert Tyrrell And Partner': ['Robert Tyrrell And Partner'],
 'Teamad Ab': ['Teamad Ab'],
 'Teeley Assets Ltd Syndicate': ['Teeley Assets Ltd Syndicate (mgr'],
 'The Dalmunzie Devils Partnership': ['The Dalmunzie Devils Partnership'],
 'The Leap Year Partnership': ['The Leap Year Partnership'],
 'The Reserved Judgment Partnership'

In [267]:
{p:corporations[p] for p in sorted({k:v for k,v in corporations.items() if len(k)==5}, key=len)[:10]}

{'Recep': ['Aller, Maria Sol, Sevim, Cem and Avsar, Recep'],
 'Llc (': ['Beach Haven Thoroughbreds, Llc ('],
 'Ltd .': ['Breeding & Racing Enterpr. Ltd .',
  'Gainsborough Stud Manag. Ltd .',
  'Kaneko Makoto Holdings Co. Ltd .',
  'North Hills Management Co. Ltd .',
  'P.H. Betts (holdings) Ltd .',
  'Ronchalon Racing (uk) Ltd .',
  'Silk Co. Ltd .',
  'Sunday Racing Co. Ltd .',
  'Yushun Horse Club Co. Ltd .'],
 'Cecil': ['Cecil And Miss Alison Wiggins'],
 'Ec. X': ['Ec. X'],
 'Esece': ['Esece'],
 'Ltd 3': ['Favourites Racing (syndication) Ltd 3'],
 'Eckel': ['Frau Dr.Cl.Eckel'],
 'Kecol': ['Kecol'],
 'Kilco': ['Kilco (international) Ltd']}

In [268]:
pb_corps=set()
for p in corporations:
    if len(p)<=4:
        pb_corps.update(corporations[p])
len(pb_corps)

500

In [274]:
pd.DataFrame({'names':list(pb_persons.union(pb_corps))}).to_csv('./data/names_to_split.csv', index=False)